This Movie Recommnedator system is Content based recommneder system

In [128]:
import numpy as np
import pandas as pd

In [129]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [130]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [131]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [132]:
movies.shape, credits.shape

((4803, 20), (4803, 4))

In [133]:
# we would like to work on a single dataset for the ease, hence we are merging the datasets
# Here, we used title column as a key column.
movies = movies.merge(credits, on='title')
movies.shape

(4809, 23)

In [134]:
# there will be some columns which we will drop, since they are of no use to us.
# following will be the columns which will be useful to us and we'll we working on
# genres, id, keywords, title, overview, cast, crew

movies = movies[['id','title','overview','genres', 'keywords',  'cast','crew']]
movies.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [135]:
movies.isnull().sum()

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [136]:
movies.dropna(inplace=True)

In [137]:
movies.duplicated().sum()

0

In [138]:
genres0 = movies.iloc[0].genres
genres0

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [139]:
# we can see that the genres is basically a string with a list in it, and we need the list
# therefore, we have the following module in python
import ast
ast.literal_eval(genres0)

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [140]:
# we are creating a function here to have a list of 'name' in genres

def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [141]:
movies['genres'].apply(convert)

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4806, dtype: object

In [142]:
movies['genres'] = movies['genres'].apply(convert)

In [143]:
movies.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [144]:
# we will do the same thing for the keyword
movies['keywords'] = movies['keywords'].apply(convert)

In [145]:
movies['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [146]:
# the convert function which we created above was to fetch the 'name' value for all the dictionaries present in that list
# but, here in 'cast', we need the name of only first three characters as they are the protagonist

def convert_cast(obj):
    l = []
    count = 0
    for i in ast.literal_eval(obj):
        if count !=3:
            l.append(i['name'])
            count+=1
        else:
            break
    return l

In [147]:
movies['cast'].apply(convert_cast)

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805         [Edward Burns, Kerry Bishé, Marsha Dietlein]
4806           [Eric Mabius, Kristin Booth, Crystal Lowe]
4807            [Daniel Henney, Eliza Coupe, Bill Paxton]
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldman]
Name: cast, Length: 4806, dtype: object

In [148]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [149]:
movies['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [150]:
# in the crew data, we can see a keyword for 'job' with value as director
# and in the same dictionary we get the name for it
# we will fetch the name of the director
def convert_crew(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [151]:
movies['crew'].apply(convert_crew)

0           [James Cameron]
1          [Gore Verbinski]
2              [Sam Mendes]
3       [Christopher Nolan]
4          [Andrew Stanton]
               ...         
4804     [Robert Rodriguez]
4805         [Edward Burns]
4806          [Scott Smith]
4807          [Daniel Hsia]
4808     [Brian Herzlinger]
Name: crew, Length: 4806, dtype: object

In [152]:
movies['crew'] = movies['crew'].apply(convert_crew)

In [153]:
# now, lets have a look at the final dataset we have
movies.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [154]:
# every column is in the form of a list, excpet the overview one. Therefore we will convert it in a list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [155]:
# since our system is content based, it works on tags
# you would have noticed that tags generally don't have space in between them
# we are applying the same logic here
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [156]:
movies.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [157]:
# now, we basically have tags stored in object columns like overview, genres, cast, crew and keywords
# therefore we can merge them to form a single column, named, tags
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [158]:
movies = movies.drop(['crew','cast','overview','keywords','genres'], axis=1)
movies.head(1)

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [161]:
movies['tags'] = movies['tags'].apply(lambda x:" ".join(x))

In [162]:
movies.head(1)

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [163]:
movies['tags'] = movies['tags'].apply(lambda x:x.lower())

In [170]:
# we will now convert our data into vector, the nearest vectos will be recommended
# vectorization was done to calculate number of common words

# now, there are words like actor/actors or action/actions, these refers to same thing and slows our algorithm
# we will converge them using nltk library

import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [171]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

In [172]:
movies['tags'] = movies['tags'].apply(stem)

In [173]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=8000, stop_words='english')
vectors = cv.fit_transform(movies['tags']).toarray()

In [174]:
# as mentioned above, we will calc the distance betweent to vectors
# we will find the cosine similarity

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
similarity.shape, similarity[0].shape

((4806, 4806), (4806,))

In [182]:
similarity

array([[1.        , 0.07352146, 0.07784989, ..., 0.04095301, 0.        ,
        0.        ],
       [0.07352146, 1.        , 0.05723638, ..., 0.02258194, 0.        ,
        0.02478408],
       [0.07784989, 0.05723638, 1.        , ..., 0.0239114 , 0.        ,
        0.        ],
       ...,
       [0.04095301, 0.02258194, 0.0239114 , ..., 1.        , 0.03809697,
        0.04141577],
       [0.        , 0.        , 0.        , ..., 0.03809697, 1.        ,
        0.0836242 ],
       [0.        , 0.02478408, 0.        , ..., 0.04141577, 0.0836242 ,
        1.        ]])

In [181]:
# since, we are working with help of index we need to have the index
list(enumerate(similarity[0]))

[(0, 0.9999999999999999),
 (1, 0.07352146220938079),
 (2, 0.07784989441615231),
 (3, 0.06320606175199697),
 (4, 0.16865480854231357),
 (5, 0.097037084956597),
 (6, 0.03419927840283847),
 (7, 0.13046561461068842),
 (8, 0.05113099925649136),
 (9, 0.0879173964527716),
 (10, 0.0879173964527716),
 (11, 0.08268982305947231),
 (12, 0.07453559924999299),
 (13, 0.03669879217087869),
 (14, 0.11785113019775792),
 (15, 0.05547001962252292),
 (16, 0.07071067811865475),
 (17, 0.12297746456210364),
 (18, 0.08664587415167274),
 (19, 0.07339758434175737),
 (20, 0.052097644344304236),
 (21, 0.0967301666813349),
 (22, 0.0563436169819011),
 (23, 0.07784989441615231),
 (24, 0.04714045207910317),
 (25, 0.045407660918649985),
 (26, 0.1363988678940947),
 (27, 0.16269784336399215),
 (28, 0.09802861627917438),
 (29, 0.05590169943749475),
 (30, 0.06030226891555273),
 (31, 0.13840913308956662),
 (32, 0.0769800358919501),
 (33, 0.08606629658238704),
 (34, 0.0),
 (35, 0.08606629658238704),
 (36, 0.15220988371246982

In [185]:
# now sorting to get the index of most similar movie, that is movies with least dist betweem them at the top
# we will have the first 5 movies as our reccommended movie
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

[(2409, 0.2504897164340598),
 (1216, 0.24693239916239743),
 (3730, 0.23333333333333334),
 (539, 0.23008949665421108),
 (507, 0.22537446792760443)]

In [180]:
# now to fetch the movie, we will use the index
# so, provided code helps to find the index
movies[movies['title'] == 'Batman Begins'].index[0]

119

In [195]:
def recommend(movie):
    movie_index = movies[movies['title']==movie].index[0]
    dist = similarity[movie_index]
    movie_list = sorted(list(enumerate(dist)), reverse=True, key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(movies.iloc[i[0]].title)

In [196]:
# testing1
recommend('Avatar')

Aliens
Aliens vs Predator: Requiem
Falcon Rising
Titan A.E.
Independence Day


In [197]:
#testing2
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf
